In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Data Loader

In [9]:
# Path to the image data folder
data_dir = "/content/drive/MyDrive/Colab Notebooks/notebook/5002 final/Data_Q2-20231214T121638Z-001/Data_Q2/train_data"

# Lists to store image paths and labels
image_paths = []
labels = []

# Dictionary for one-hot encoding of labels
label_map = {'Sunny': [1, 0, 0, 0, 0], 'Snowy': [0, 1, 0, 0, 0],
             'Cloudy': [0, 0, 1, 0, 0], 'Rainy': [0, 0, 0, 1, 0],
             'Foggy': [0, 0, 0, 0, 1]}

# Load the dataset
for filename in os.listdir(data_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(data_dir, filename)
        # Extracting label from the filename
        label = ''.join([i for i in filename if not i.isdigit()]).rstrip('.jpg').rstrip('.png').rstrip('_')
        if label in label_map:
            image_paths.append(image_path)
            labels.append(label_map[label])
        else:
            print(f"Label not recognized in file: {filename}")

if not image_paths:
    raise ValueError("No images found in the data directory.")

# Transformation for images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom dataset function
def custom_dataset(root_dir, paths, labels, transform):
    for path, label in zip(paths, labels):
        image = Image.open(path)
        image = transform(image)
        yield image, torch.tensor(label)

# Creating the dataset
dataset = list(custom_dataset(data_dir, image_paths, labels, transform))

# DataLoader
batch_size = 8
# DataLoader with shuffle: The order of data loading will be randomly shuffled in each training epoch, helping to improve model generalization.
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [10]:
dataloader

In [11]:
print("Number of images loaded:", len(image_paths))

Number of images loaded: 250


# Model

In [14]:
# Define network layers and parameters
conv1 = torch.nn.Conv2d(3, 32, 3, stride=1, padding=1)  # Input channels: 3, Output channels: 32
pool = torch.nn.MaxPool2d(2, 2)                        # Pooling layer
conv2 = torch.nn.Conv2d(32, 64, 3, stride=1, padding=1) # Input channels: 32, Output channels: 64
fc1 = torch.nn.Linear(64 * 64 * 64, 128)                # First fully connected layer
fc2 = torch.nn.Linear(128, 5)                           # Second fully connected layer, outputting 5 classes

# Optimizer and loss function
optimizer = optim.Adam([{'params': conv1.parameters()}, {'params': conv2.parameters()}, {'params': fc1.parameters()}, {'params': fc2.parameters()}], lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        # Reset gradients
        optimizer.zero_grad()

        # Forward pass
        x = pool(F.relu(conv1(images)))
        x = pool(F.relu(conv2(x)))
        x = x.view(-1, 64 * 64 * 64)  # Flatten
        x = F.relu(fc1(x))
        x = fc2(x)

        # Calculate loss and perform backpropagation
        loss = criterion(x, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader)}")

print("Training complete")

# Calculate accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        x = pool(F.relu(conv1(images)))
        x = pool(F.relu(conv2(x)))
        x = x.view(-1, 64 * 64 * 64)
        x = F.relu(fc1(x))
        x = fc2(x)
        _, predicted = torch.max(x.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.max(labels, 1)[1]).sum().item()

print(f"Accuracy on the training set: {100 * correct / total}%")


Epoch 1/50, Loss: 4.635122068226337
Epoch 2/50, Loss: 1.1784597299993038
Epoch 3/50, Loss: 0.5277210059575737
Epoch 4/50, Loss: 0.18277416107594036
Epoch 5/50, Loss: 0.08107339651905932
Epoch 6/50, Loss: 0.11311018998094369
Epoch 7/50, Loss: 0.054531371730263345
Epoch 8/50, Loss: 0.04591831673997149
Epoch 9/50, Loss: 0.08271387098284322
Epoch 10/50, Loss: 0.029839536633517127
Epoch 11/50, Loss: 0.027495925307448488
Epoch 12/50, Loss: 0.009727693596687459
Epoch 13/50, Loss: 0.016071633984211076
Epoch 14/50, Loss: 0.015644576378690545
Epoch 15/50, Loss: 0.009847133841049072
Epoch 16/50, Loss: 0.009699244988723876
Epoch 17/50, Loss: 0.010271459474552103
Epoch 18/50, Loss: 0.01981860806381519
Epoch 19/50, Loss: 0.022306660712274606
Epoch 20/50, Loss: 0.02130480974483362
Epoch 21/50, Loss: 0.023283880282406244
Epoch 22/50, Loss: 0.01785661920439452
Epoch 23/50, Loss: 0.006606421885408054
Epoch 24/50, Loss: 0.006898822162156648
Epoch 25/50, Loss: 0.008183515012746057
Epoch 26/50, Loss: 0.007